# End-to-End Demonstration: Canvas

*Goal*: run the Canvas Extractor and then upload the results into an `LMS` database.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.
1. Please read through the notebook and fill in the two text boxes below before executing the entire notebook.

In [ ]:
# Load some utilities
from IPython.display import display, Markdown
import ipywidgets as widgets

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# This logging produces too much information in the underlying Canvas API package, 
# leading to personally-identifiable information in the logs. The following configuration
# reduces that part of the logging configuration back to the WARNING level.
canvasapi_logger = logging.getLogger("canvasapi")
if canvasapi_logger.getEffectiveLevel() < logging.WARN:
    canvasapi_logger.setLevel(logging.WARN)

## Prepare Input Data

Update the variables in the next block as needed.

In [ ]:
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "c-data"
SYNC_DATABASE_DIRECTORY=OUTPUT_DIRECTORY
START_DATE="2021-02-01"
END_DATE="2021-07-28"
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433
EXTRACT_ASSIGNMENTS = True
EXTRACT_ACTIVITIES = False
EXTRACT_ATTENDANCE = False
EXTRACT_GRADES = False

Run the next cell to create text input boxes for providing the agency's Canvas URL and a Canvas access token. [Instructions for acquiring an access token](https://community.canvaslms.com/t5/Student-Guide/How-do-I-manage-API-access-tokens-as-a-student/ta-p/273). Then run the next cell down to capture the values before running the last cells.

In [ ]:
def get_access_token(token, base_url):
    return token, base_url

token_widget = widgets.Text(
    description="Access Token"
)
base_url_widget = widgets.Text(
    description="Base URL"
)
result = widgets.interactive(get_access_token, token=token_widget, base_url=base_url_widget)
display(result)

In [ ]:
TOKEN = result.children[0].value
CANVAS_BASE_URL = result.children[1].value
print("Captured token and base URL.")

## Run the Canvas Extractor

In [ ]:
from edfi_canvas_extractor.helpers.arg_parser import MainArguments as c_args
from edfi_canvas_extractor import extract_facade

arguments = c_args(
    base_url=CANVAS_BASE_URL,
    access_token=TOKEN,
    output_directory=OUTPUT_DIRECTORY,
    log_level=LOG_LEVEL,
    start_date=START_DATE,
    end_date=END_DATE,
    sync_database_directory=SYNC_DATABASE_DIRECTORY,
    extract_assignments=EXTRACT_ASSIGNMENTS,
    extract_activities=EXTRACT_ACTIVITIES,
    extract_attendance=EXTRACT_ATTENDANCE,
    extract_grades=EXTRACT_GRADES,
)

extract_facade.run(arguments)

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [ ]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL
)
arguments.set_connection_string_using_integrated_security(
    DB_SERVER,
    DB_PORT,
    DB_NAME,
)
# For password auth, comment out the line above and uncomment this one:
# arguments.set_connection_string(
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
#     USERNAME,
#     PASSWORD,
# )

loader_facade.run_loader(arguments)